# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p sc:name?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-cda4aeaa8e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Presidents of countries") 

Consider the following exploratory information need:

> You investigating presidents of the republic, or similar roles, across international states around the world

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q248577`  | President of the republic | node      |
| `wd:Q11696`   | President of U.S.A.      | node      |
| `wd:Q332711`  | President of Italy        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q30`      | U.S.A.        | node |



Also consider

```
?p wdt:P39/wdt:P279* wd:Q248577  .  
```

is the BGP to retrieve all **presidents of the world countries through history**

## Workload Goals

1. Identify the BGP for obtaining important attributes for people, e.g., date of birth/death, gender, profession 

2. Identify the BGP to retrieve countries with  had at least once had a president

3. When was the first president of each country born? How many presidents each country had?

4. Is there a country that had at some point a woman as a president?

5. Analyze the number of presidents per country through time
 
   5.1 What are the top-5 countries for number of presidents? Which countries had the least?
   
   5.2 For how many presidents we know a date of death in each country?
   
   5.3 Which are the professions of different presidents? How many presidents had a specific profession?


In [2]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P39/wdt:P279* wd:Q248577  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1649')]


1

## Task 1

Looking for subclass of president of republic and find the property

In [3]:
queryString = """
SELECT DISTINCT ?pr ?name 
WHERE { 

?pr wdt:P279+ wd:Q248577 .
?pr sc:name?name .

} 
LIMIT 200
"""

print("Results")
run_query(queryString)



Results
[('pr', 'http://www.wikidata.org/entity/Q14946265'), ('name', 'President of Bulgaria')]
[('pr', 'http://www.wikidata.org/entity/Q1054799'), ('name', 'President of the Republic of Poland')]
[('pr', 'http://www.wikidata.org/entity/Q1625960'), ('name', 'President of Albania')]
[('pr', 'http://www.wikidata.org/entity/Q18204367'), ('name', 'President of Uganda')]
[('pr', 'http://www.wikidata.org/entity/Q19161338'), ('name', 'President of Madagascar')]
[('pr', 'http://www.wikidata.org/entity/Q2672728'), ('name', 'President of Ghana')]
[('pr', 'http://www.wikidata.org/entity/Q268984'), ('name', 'president of Croatia')]
[('pr', 'http://www.wikidata.org/entity/Q27977064'), ('name', 'President of Costa Rica')]
[('pr', 'http://www.wikidata.org/entity/Q3409186'), ('name', 'President of Ethiopia')]
[('pr', 'http://www.wikidata.org/entity/Q3409199'), ('name', 'President of the Democratic Republic of the Congo')]
[('pr', 'http://www.wikidata.org/entity/Q495877'), ('name', 'President of Sloven

107

In [4]:
#look for the properties
queryString = """
SELECT DISTINCT ?pr ?name 
WHERE { 

?p wdt:P279* wd:Q248577 ;
     ?pr ?o.
?pr sc:name?name .

} 
LIMIT 200
"""

print("Results")
run_query(queryString)




Results
[('pr', 'http://www.wikidata.org/prop/direct/P2389'), ('name', 'organization directed by the office or position')]
[('pr', 'http://www.wikidata.org/prop/direct/P2098'), ('name', 'substitute/deputy/replacement of office/officeholder')]
[('pr', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('pr', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('pr', 'http://www.wikidata.org/

92

Find the class of human and find the relations of interest (probably the literal once).

In [5]:
queryString = """
SELECT DISTINCT ?ist ?istance ?pr ?name
WHERE { 

?p wdt:P39 wd:Q332711 ;
    wdt:P31 ?ist ;
    ?pr ?o .
?pr sc:name?name .
?ist sc:name?istance .
FILTER (REGEX(?name, \".*date.*\") || REGEX(?name, \".*gen.*\") || REGEX(?name, \".*birth.*\") || REGEX(?name, \".*death.*\") || REGEX(?name, \".*prof.*\") || REGEX(?name, \".*occ.*\"))

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P2600'), ('name', 'Geni.com profile ID')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P803'), ('name', 'professorship')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death')]
[('ist', 'http://www.wikidata.org/entity/Q5'), ('istance', 'human'), ('pr', 'http://www.wikidata

10

- `wd:Q5` class of human
- `wdt:P569` date of birth
- `wdt:P570` date of death
- `wdt:P21` gender
- `wdt:P106` occupation

Looking for a superclass of human to find a person class

In [7]:
queryString = """
SELECT ?sup ?name
WHERE { 

wd:Q5 wdt:P279 ?sup.

?sup sc:name?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('sup', 'http://www.wikidata.org/entity/Q154954'), ('name', 'natural person')]
[('sup', 'http://www.wikidata.org/entity/Q164509'), ('name', 'omnivore')]


3

Found `wd:Q215627` the class of person. Now look for some relation

In [8]:
queryString = """
SELECT DISTINCT ?ist ?istance ?pr ?name
WHERE { 

?p wdt:P31 wd:Q215627 ;
       ?pr ?o .
?pr sc:name?name .
#?ist sc:name?istance .
FILTER (REGEX(?name, \".*date.*\") || REGEX(?name, \".*gen.*\") || REGEX(?name, \".*birth.*\") || REGEX(?name, \".*death.*\") || REGEX(?name, \".*prof.*\") || REGEX(?name, \".*occ.*\"))
#FILTER (!isLiteral(?o))
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P2600'), ('name', 'Geni.com profile ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P6634'), ('name', 'LinkedIn personal profile ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('pr', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('pr', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death')]
[('pr', 'http://www.wikidata.org/prop/direct/P1477'), ('name', 'birth name')]
[('pr', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('pr', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('pr', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('pr', 'http://www.wikidata.org/prop/direct/P3544'), ('name', 'Te Papa agent ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
[('pr', 'http:/

13

Predicate of interest

- `wd:Q215627` class of person
- `wdt:P569` date of birth
- `wdt:P570` date of death
- `wdt:P136` genre
- `wdt:P21` sex or gender
- `wdt:P106` occupation

Looking for all person and subclasses (probably it make sense to look only for the class person).

The death is option because I want information also to people still alive.

In [9]:
queryString = """
SELECT ?name ?gender ?birth ?death ?occupation
WHERE { 

?p wdt:P31 wd:Q5 .
?p wdt:P21 ?g .
?p wdt:P569 ?birth .
OPTIONAL{ ?p wdt:P570 ?death .}
?p wdt:P106 ?o .
?p sc:name?name .
?g sc:name?gender .
?o sc:name?occupation .

}

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('name', 'Mateo Egea'), ('gender', 'male'), ('birth', '1896-01-01T00:00:00Z'), ('death', '1979-01-01T00:00:00Z'), ('occupation', 'politician')]
[('name', 'José Ximénez'), ('gender', 'male'), ('birth', '1601-01-01T00:00:00Z'), ('death', '1672-08-09T00:00:00Z'), ('occupation', 'composer')]
[('name', 'José Ximénez'), ('gender', 'male'), ('birth', '1601-01-01T00:00:00Z'), ('death', '1672-08-09T00:00:00Z'), ('occupation', 'organist')]
[('name', 'Empress Dowager Longyu'), ('gender', 'female'), ('birth', '1868-01-28T00:00:00Z'), ('death', '1913-02-22T00:00:00Z'), ('occupation', 'politician')]
[('name', 'Eugénio de Castro'), ('gender', 'male'), ('birth', '1869-03-04T00:00:00Z'), ('death', '1944-08-17T00:00:00Z'), ('occupation', 'pedagogue')]
[('name', 'Eugénio de Castro'), ('gender', 'male'), ('birth', '1869-03-04T00:00:00Z'), ('death', '1944-08-17T00:00:00Z'), ('occupation', 'writer')]
[('name', 'Eugénio de Castro'), ('gender', 'male'), ('birth', '1869-03-04T00:00:00Z'), ('death', '1

10

The result above shows as many rows for each person as how many different occupation he has/had.

To solve this I group all the occupations in one field with the GROUP_CONCAT operation.

In [10]:
queryString = """
SELECT ?name ?gender ?birth ?death (GROUP_CONCAT(?occupation;separator=';') AS ?occ)
WHERE { 

?p wdt:P31 wd:Q5 .
?p wdt:P21 ?g .
?p wdt:P569 ?birth .
OPTIONAL{ ?p wdt:P570 ?death .}
?p wdt:P106 ?o .
?p sc:name?name .
?g sc:name?gender .
?o sc:name?occupation .

}
GROUP BY ?name ?gender ?birth ?death
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('name', 'Henri Sannier'), ('gender', 'male'), ('birth', '1947-09-07T00:00:00Z'), ('occ', 'journalist;politician;anciens cadres;presenter;news presenter')]
[('name', 'Peter Futej'), ('gender', 'male'), ('birth', '1970-06-14T00:00:00Z'), ('occ', 'association football player')]
[('name', 'Andrzej Paluchowski'), ('gender', 'male'), ('birth', '1933-01-01T00:00:00Z'), ('death', '2017-01-01T00:00:00Z'), ('occ', 'philologist;historian')]
[('name', 'Jean-Baptiste Émile Dropsy'), ('gender', 'male'), ('birth', '1848-01-01T00:00:00Z'), ('death', '1923-01-01T00:00:00Z'), ('occ', 'medalist')]
[('name', 'John Christensen'), ('gender', 'male'), ('birth', '1962-02-20T00:00:00Z'), ('death', '2009-03-31T00:00:00Z'), ('occ', 'athlete')]
[('name', 'Rod Dreher'), ('gender', 'male'), ('birth', '1967-02-14T00:00:00Z'), ('occ', 'journalist')]
[('name', 'Nikolaus Weselik'), ('gender', 'male'), ('birth', '1966-02-04T00:00:00Z'), ('occ', 'writer;lawyer')]
[('name', 'Clarence Heise'), ('gender', 'male'),

10

## Task 2 
Identify the BGP to retrieve countries with had at least once had a president

First find the class of country starting from Italy and find the instance of the class

In [11]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
wd:Q38 wdt:P31 ?p.

?p sc:name?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('p', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]


2

`wd:Q6256` is the class of country. 

Looking for relations that associate a people that held the role of president to something that belong at the class of the country

In [12]:
queryString = """
SELECT DISTINCT ?p ?name

WHERE { 

?a wdt:P39/wdt:P279* wd:Q248577 .
 ?o ?p ?a ;
    wdt:P31 wd:Q6256.

?p sc:name?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P35'), ('name', 'head of state')]
[('p', 'http://www.wikidata.org/prop/direct/P6'), ('name', 'head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]


3

Looking at the relation "head of state" `wdt:P35` with Italy to understand what it means.

In [13]:
queryString = """
SELECT ?p ?name ?country ?people

WHERE { 

?a wdt:P39/wdt:P279* wd:Q248577 .
 ?o ?p ?a ;
    wdt:P31 wd:Q6256.

?p sc:name?name .
?o sc:name?country .
?a sc:name?people .
FILTER (REGEX(?country,\".*taly.*\"))
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P35'), ('name', 'head of state'), ('country', 'Italy'), ('people', 'Sergio Mattarella')]


1

Understood that this is the current president of the republic

So, since the sublcasses of the President of Republic have a relation named country (`P17`) I look to that.

Find if a country had at least one president starting by all people that held the role of president and looking at the country of the subclass of President of Republic.

In [14]:
queryString = """
SELECT DISTINCT ?country
WHERE { 

?p wdt:P279+ wd:Q248577  . 
?a wdt:P39 ?p.
?p wdt:P17 ?c.
?c sc:name?country .

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('country', 'Poland')]
[('country', 'Albania')]
[('country', 'Uganda')]
[('country', 'Madagascar')]
[('country', 'Ghana')]
[('country', 'Croatia')]
[('country', 'Costa Rica')]
[('country', 'Ethiopia')]
[('country', 'Democratic Republic of the Congo')]
[('country', 'Slovenia')]
[('country', 'Guatemala')]
[('country', 'Namibia')]
[('country', 'Syria')]
[('country', 'Tanzania')]
[('country', 'Mongolia')]
[('country', 'Romania')]
[('country', 'El Salvador')]
[('country', 'Lithuania')]
[('country', 'Iraq')]
[('country', 'Philippines')]
[('country', 'Pakistan')]
[('country', 'Peru')]
[('country', 'Confederate States of America')]
[('country', 'Sri Lanka')]
[('country', 'Haiti')]
[('country', 'Singapore')]
[('country', 'Hungary')]
[('country', 'Republic of China 1912–1949')]
[('country', 'Georgia')]
[('country', 'Taiwan')]
[('country', 'Somalia')]
[('country', 'Zambia')]
[('country', 'São Tomé and Príncipe')]
[('country', 'Iceland')]
[('country', 'Moldova')]
[('country', 'Seychelles'

99

## Task 3
When was the first president of each country born? How many presidents each country had?

From one of the first query about the relations of the subproperty of the president of republic I found the "start time" property, which has code `wdt:P580` and also age of candidacy (`wdt:P2998`)

In [7]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?p wdt:P279+ wd:Q248577  . 
?a wdt:P39 ?p;
    ?pr ?p.
?pr sc:name ?prop .

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('prop', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P39'), ('prop', 'position held')]


3

In [8]:
#look for some list
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?p wdt:P279+ wd:Q248577  .
?p ?pr ?x.
?pr sc:name?prop .
filter(REGEX(?prop,'.*list.*'))
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1754'), ('prop', 'category related to list')]
[('pr', 'http://www.wikidata.org/prop/direct/P2354'), ('prop', 'has list')]
[('pr', 'http://www.wikidata.org/prop/direct/P360'), ('prop', 'is a list of')]
[('pr', 'http://www.wikidata.org/prop/direct/P5008'), ('prop', 'on focus list of Wikimedia project')]


4

With `wdt:P2354` I have the list of the president for each different country (maybe)

In [9]:
#look for the president of the republic of Italy
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?pr wdt:P31 wd:Q5;
    wdt:P39 wd:Q332711.
?pr sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q1220'), ('prop', 'Giorgio Napolitano')]
[('pr', 'http://www.wikidata.org/entity/Q1224'), ('prop', 'Carlo Azeglio Ciampi')]
[('pr', 'http://www.wikidata.org/entity/Q1228'), ('prop', 'Oscar Luigi Scalfaro')]
[('pr', 'http://www.wikidata.org/entity/Q1241'), ('prop', 'Giovanni Gronchi')]
[('pr', 'http://www.wikidata.org/entity/Q1230'), ('prop', 'Francesco Cossiga')]
[('pr', 'http://www.wikidata.org/entity/Q1238'), ('prop', 'Giuseppe Saragat')]
[('pr', 'http://www.wikidata.org/entity/Q1245'), ('prop', 'Enrico de Nicola')]
[('pr', 'http://www.wikidata.org/entity/Q1233'), ('prop', 'Sandro Pertini')]
[('pr', 'http://www.wikidata.org/entity/Q3956186'), ('prop', 'Sergio Mattarella')]
[('pr', 'http://www.wikidata.org/entity/Q1242'), ('prop', 'Luigi Einaudi')]
[('pr', 'http://www.wikidata.org/entity/Q1235'), ('prop', 'Giovanni Leone')]
[('pr', 'http://www.wikidata.org/entity/Q1240'), ('prop', 'Antonio Segni')]


12

In [10]:
#look for the property of the list
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?p wdt:P279* wd:Q248577  .
?p wdt:P2354 ?x.
?x ?pr ?y  .

?pr sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1001'), ('prop', 'applies to jurisdiction')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('prop', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('prop', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P158'), ('prop', 'seal image')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('prop', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('prop', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P1754'), ('prop', 'category related to list')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('prop', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P1814'), ('prop', 'name in kana')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('prop', 'Google Knowledge Graph ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('prop', 'instance of')]
[('pr', 'http

26

In [12]:
#look for some other properties between people that held the same position
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?b wdt:P31 wd:Q5.
?a ?pr ?b.
?pr sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('prop', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P22'), ('prop', 'father')]
[('pr', 'http://www.wikidata.org/prop/direct/P25'), ('prop', 'mother')]
[('pr', 'http://www.wikidata.org/prop/direct/P26'), ('prop', 'spouse')]
[('pr', 'http://www.wikidata.org/prop/direct/P3373'), ('prop', 'sibling')]
[('pr', 'http://www.wikidata.org/prop/direct/P40'), ('prop', 'child')]
[('pr', 'http://www.wikidata.org/prop/direct/P157'), ('prop', 'killed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P3342'), ('prop', 'significant person')]
[('pr', 'http://www.wikidata.org/prop/direct/P737'), ('prop', 'influenced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1327'), ('prop', 'partner in business or sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P451'), ('prop', 'unmarried partner')]
[('pr', 'http://www.wikidata.org/prop/direct/P3448

29

In [13]:
#look for some other properties between people that held the same position
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?b wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?pr ?b.
?pr sc:name?prop .
filter(?a != ?b)
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('prop', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('prop', 'employer')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P22'), ('prop', 'father')]
[('pr', 'http://www.wikidata.org/prop/direct/P25'), ('prop', 'mother')]
[('pr', 'http://www.wikidata.org/prop/direct/P26'), ('prop', 'spouse')]
[('pr', 'http://www.wikidata.org/prop/direct/P40'), ('prop', 'child')]
[('pr', 'http://www.wikidata.org/prop/direct/P737'), ('prop', 'influenced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P3373'), ('prop', 'sibling')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('prop', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P157'), ('prop', 'killed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1366'), ('prop', 'replaced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('prop', 'followed by')]


13

In [21]:
queryString = """
SELECT DISTINCT ?aP ?bP
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?b wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a wdt:P156|wdt:P1366|wdt:1356 ?b.
?a sc:name?aP .
?b sc:name?bP .
filter(?a != ?b)
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('aP', 'Hafez al-Assad'), ('bP', 'Bashar al-Assad')]
[('aP', 'Akwasi Afrifa'), ('bP', 'Nii Amaa Ollennu')]


2

None of these property tell me how two people are related...

In [14]:
#look for some objects that relate people and the position President of the Republic
queryString = """
SELECT DISTINCT ?o ?obj
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX ?o.
?p ?prY ?o.

?o sc:name?obj .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q19689'), ('obj', 'Tirana')]
[('o', 'http://www.wikidata.org/entity/Q35525'), ('obj', 'White House')]
[('o', 'http://www.wikidata.org/entity/Q40362'), ('obj', 'Sahrawi Arab Democratic Republic')]
[('o', 'http://www.wikidata.org/entity/Q907112'), ('obj', 'Transnistria')]
[('o', 'http://www.wikidata.org/entity/Q2363110'), ('obj', 'Deputy President of South Africa')]
[('o', 'http://www.wikidata.org/entity/Q1641162'), ('obj', 'Vice President of India')]
[('o', 'http://www.wikidata.org/entity/Q57416'), ('obj', 'Joseph Kabila')]
[('o', 'http://www.wikidata.org/entity/Q2723485'), ('obj', 'Vice President of the Philippines')]
[('o', 'http://www.wikidata.org/entity/Q677555'), ('obj', 'Rashtrapati Bhavan')]
[('o', 'http://www.wikidata.org/entity/Q707830'), ('obj', 'Vice President of the Republic of China')]
[('o', 'http://www.wikidata.org/entity/Q878151'), ('obj', 'President of El Salvador')]
[('o', 'http://www.wikidata.org/entity/Q223079'), ('obj',

50

Look for example to the president of US `wd:Q1169`

In [23]:
queryString = """
SELECT DISTINCT ?prY ?obj
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX wd:Q11696.
?p ?prY wd:Q11696.
#?prX sc:name?obj .
?prY sc:name?obj .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('prY', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'subclass of')]


1

So the President of the US has a subclass. What is this??

In [15]:
queryString = """
SELECT DISTINCT ?p ?obj
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX wd:Q11696.
?p wdt:P279 wd:Q11696.
?p sc:name?obj .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1467287'), ('obj', 'President-elect of the United States')]


1

Hoping that this solve the question...

In [25]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX wd:Q11696.
?p wdt:P279 wd:Q11696.
#look at the property outcoming
?p ?pr ?o.
?pr sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1001'), ('prop', 'applies to jurisdiction')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('prop', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('prop', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P279'), ('prop', 'subclass of')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('prop', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P373'), ('prop', 'Commons category')]
[('pr', 'http://www.wikidata.org/prop/direct/P571'), ('prop', 'inception')]
[('pr', 'http://www.wikidata.org/prop/direct/P646'), ('prop', 'Freebase ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P828'), ('prop', 'has cause')]
[('pr', 'http://www.wikidata.org/prop/direct/P511'), ('prop', 'honorific prefix')]
[('pr', 'http://www.wikidata.org/prop/direct/P1308'), ('prop', 'officeholder')]


11

In [26]:
queryString = """
SELECT DISTINCT  ?p ?prop ?o
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX wd:Q11696.
?p wdt:P279 wd:Q11696.
#look at the property outcoming
?p wdt:P571 ?o.
?p sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1467287'), ('prop', 'President-elect of the United States'), ('o', '1789-01-01T00:00:00Z')]


1

In [27]:
queryString = """
SELECT DISTINCT  ?pr ?prop
WHERE { 
?p wdt:P279* wd:Q248577  .
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?a ?prX wd:Q11696.
?p wdt:P279 wd:Q11696.
#look at the property outcoming
?o ?pr ?p.
?pr sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P39'), ('prop', 'position held')]


1

In [28]:
#subclass???

queryString = """
SELECT DISTINCT ?x ?prop
WHERE { 
?p wdt:P279 wd:Q248577 .
?a wdt:P39 ?p.
?x wdt:P279 ?p.
?x sc:name?prop .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q55831'), ('prop', 'Acting President of Poland')]
[('x', 'http://www.wikidata.org/entity/Q68390254'), ('prop', 'Jefe Supremo del Estado de El Salvador')]
[('x', 'http://www.wikidata.org/entity/Q4676854'), ('prop', 'Acting President of Pakistan')]
[('x', 'http://www.wikidata.org/entity/Q1467287'), ('prop', 'President-elect of the United States')]
[('x', 'http://www.wikidata.org/entity/Q2123628'), ('prop', 'Acting President of the United States')]


5

In [29]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 
?p wdt:P279 wd:Q248577 .
?a wdt:P39 ?p.
?p ?pr ?o.
?pr sc:name?prop .
filter(isLiteral(?o)).

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P571'), ('prop', 'inception')]
[('pr', 'http://www.wikidata.org/prop/direct/P646'), ('prop', 'Freebase ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P373'), ('prop', 'Commons category')]
[('pr', 'http://www.wikidata.org/prop/direct/P935'), ('prop', 'Commons gallery')]
[('pr', 'http://www.wikidata.org/prop/direct/P214'), ('prop', 'VIAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2097'), ('prop', 'term length of office')]
[('pr', 'http://www.wikidata.org/prop/direct/P7859'), ('prop', 'WorldCat Identities ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('prop', 'Google Knowledge Graph ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('prop', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2397'), ('prop', 'YouTube channel ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P8687'), ('prop', 'social media followers')]
[('pr', 'http://www.wikidata.org/prop/direct/P2003'), ('prop', 'Instagra

47

In [30]:
queryString = """
SELECT DISTINCT ?p ?prop ?o
WHERE { 
?p wdt:P279 wd:Q248577 .
?a wdt:P39 ?p.
?p wdt:P580 ?o.

?p sc:name?prop .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q878222'), ('prop', 'President of the Republic of Lithuania'), ('o', '1922-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q866680'), ('prop', 'President of Vietnam'), ('o', '1976-07-02T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q5176750'), ('prop', 'President of Brazil'), ('o', '1889-11-15T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q878217'), ('prop', 'President of Vanuatu'), ('o', '1980-07-30T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q887124'), ('prop', 'President of Myanmar'), ('o', '1948-01-04T00:00:00Z')]


5

In [31]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?x wdt:P31 ?pr.
?pr sc:name?prop .
filter(REGEX(?prop,\".*occupation.*\"))

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q380141'), ('prop', 'occupational therapy')]
[('pr', 'http://www.wikidata.org/entity/Q628764'), ('prop', 'occupational medicine')]
[('pr', 'http://www.wikidata.org/entity/Q11773926'), ('prop', 'ecclesiastical occupation')]
[('pr', 'http://www.wikidata.org/entity/Q21074597'), ('prop', 'occupational carcinogen')]
[('pr', 'http://www.wikidata.org/entity/Q4220920'), ('prop', 'filmmaking occupation')]
[('pr', 'http://www.wikidata.org/entity/Q66666236'), ('prop', 'Wikimedia list of persons by occupation')]
[('pr', 'http://www.wikidata.org/entity/Q67184216'), ('prop', 'Wikimedia list of persons by gender (P21) and occupation (P106)')]
[('pr', 'http://www.wikidata.org/entity/Q63188808'), ('prop', 'Catholic religious occupation')]
[('pr', 'http://www.wikidata.org/entity/Q63187384'), ('prop', 'Jewish religious occupation')]
[('pr', 'http://www.wikidata.org/entity/Q17072363'), ('prop', 'occupational exposure limit')]
[('pr', 'http://www.wikidata.org

38

In [16]:
queryString = """
SELECT DISTINCT ?o ?prop

WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?a ?x ?o.
?o ?y ?p.
?a sc:name ?people .
?o sc:name ?prop .
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q554259'), ('prop', 'José Miguel Gómez')]
[('o', 'http://www.wikidata.org/entity/Q135547'), ('prop', 'Laurent-Désiré Kabila')]
[('o', 'http://www.wikidata.org/entity/Q312539'), ('prop', 'Diosdado Macapagal')]
[('o', 'http://www.wikidata.org/entity/Q55645704'), ('prop', 'Government of Ecuador')]
[('o', 'http://www.wikidata.org/entity/Q11816'), ('prop', 'John Quincy Adams')]
[('o', 'http://www.wikidata.org/entity/Q11896'), ('prop', 'Zachary Taylor')]
[('o', 'http://www.wikidata.org/entity/Q1333'), ('prop', 'Francisco Antonio Pinto')]
[('o', 'http://www.wikidata.org/entity/Q177182'), ('prop', 'Sixto Durán-Ballén')]
[('o', 'http://www.wikidata.org/entity/Q878835'), ('prop', 'Miguel Mariano Gómez')]
[('o', 'http://www.wikidata.org/entity/Q1480'), ('prop', 'Corazon Aquino')]
[('o', 'http://www.wikidata.org/entity/Q1464'), ('prop', 'Benigno Aquino III')]
[('o', 'http://www.wikidata.org/entity/Q57416'), ('prop', 'Joseph Kabila')]
[('o', 'http://ww

200

In [33]:
queryString = """
SELECT DISTINCT ?y ?propY

WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?a ?x ?o.
?o ?y ?p.
?y sc:name?propY .
} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/prop/direct/P1313'), ('propY', 'office held by head of government')]
[('y', 'http://www.wikidata.org/prop/direct/P39'), ('propY', 'position held')]
[('y', 'http://www.wikidata.org/prop/direct/P1889'), ('propY', 'different from')]
[('y', 'http://www.wikidata.org/prop/direct/P1906'), ('propY', 'office held by head of state')]
[('y', 'http://www.wikidata.org/prop/direct/P279'), ('propY', 'subclass of')]
[('y', 'http://www.wikidata.org/prop/direct/P1366'), ('propY', 'replaced by')]
[('y', 'http://www.wikidata.org/prop/direct/P138'), ('propY', 'named after')]
[('y', 'http://www.wikidata.org/prop/direct/P156'), ('propY', 'followed by')]
[('y', 'http://www.wikidata.org/prop/direct/P35'), ('propY', 'head of state')]
[('y', 'http://www.wikidata.org/prop/direct/P360'), ('propY', 'is a list of')]
[('y', 'http://www.wikidata.org/prop/direct/P106'), ('propY', 'occupation')]
[('y', 'http://www.wikidata.org/prop/direct/P26'), ('propY', 'spouse')]
[('y', 'http:/

17

My ideas is finishing..
Think better about reification. 

In [34]:
queryString = """
SELECT DISTINCT ?x ?propX ?y ?propY

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z ?x ?a.
?z ?y ?p.
?x sc:name?propX .
?y sc:name?propY .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/prop/direct/P35'), ('propX', 'head of state'), ('y', 'http://www.wikidata.org/prop/direct/P1906'), ('propY', 'office held by head of state')]
[('x', 'http://www.wikidata.org/prop/direct/P35'), ('propX', 'head of state'), ('y', 'http://www.wikidata.org/prop/direct/P1313'), ('propY', 'office held by head of government')]
[('x', 'http://www.wikidata.org/prop/direct/P6'), ('propX', 'head of government'), ('y', 'http://www.wikidata.org/prop/direct/P1906'), ('propY', 'office held by head of state')]
[('x', 'http://www.wikidata.org/prop/direct/P6'), ('propX', 'head of government'), ('y', 'http://www.wikidata.org/prop/direct/P1313'), ('propY', 'office held by head of government')]
[('x', 'http://www.wikidata.org/prop/direct/P1308'), ('propX', 'officeholder'), ('y', 'http://www.wikidata.org/prop/direct/P360'), ('propY', 'is a list of')]
[('x', 'http://www.wikidata.org/prop/direct/P22'), ('propX', 'father'), ('y', 'http://www.wikidata.org/prop/direct/P39')

41

Look for candidate (`wdt:P726`) and successful candidate (`wdt:P991`)!

In [35]:
queryString = """
SELECT DISTINCT ?z ?obj

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?y ?p.
?z sc:name?obj .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('z', 'http://www.wikidata.org/entity/Q174492'), ('obj', '1836 United States presidential election')]
[('z', 'http://www.wikidata.org/entity/Q534192'), ('obj', '1832 United States presidential election')]
[('z', 'http://www.wikidata.org/entity/Q14957122'), ('obj', '2008 United States presidential election in Washington (state)')]
[('z', 'http://www.wikidata.org/entity/Q16197803'), ('obj', '1988 United States presidential election in Virginia')]
[('z', 'http://www.wikidata.org/entity/Q17006707'), ('obj', '1984 United States presidential election in Georgia')]
[('z', 'http://www.wikidata.org/entity/Q17006719'), ('obj', '1988 United States presidential election in Georgia')]
[('z', 'http://www.wikidata.org/entity/Q17007083'), ('obj', '1984 United States presidential election in Delaware')]
[('z', 'http://www.wikidata.org/entity/Q17007096'), ('obj', '1988 United States presidential election in Delaware')]
[('z', 'http://www.wikidata.org/entity/Q17016038'), ('obj', '1904 United Sta

50

In [36]:
queryString = """
SELECT DISTINCT ?z ?obj

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?y ?p.
?z sc:name?obj .
filter(regex(?obj,".*talian.*"))
} 
order by ?obj
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('z', 'http://www.wikidata.org/entity/Q3722096'), ('obj', '1948 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3722097'), ('obj', '1955 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3722098'), ('obj', '1962 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3722099'), ('obj', '1964 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3586603'), ('obj', '1971 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3586604'), ('obj', '1978 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3586605'), ('obj', '1985 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3586607'), ('obj', '1992 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q3586608'), ('obj', '1999 Italian presidential election')]
[('z', 'http://www.wikidata.org/entity/Q588041'), ('obj', '2006 Italian presidential election')]
[('z', 'http:

12

I found a way to have all the president elected in time, now try to take the first president elected.

In [17]:
#look for some properties
queryString = """
SELECT DISTINCT ?pr ?prop

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?y ?p.
?z ?pr ?o.
?pr sc:name?prop .

}
ORDER BY ?prop
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P9629'), ('prop', 'Armeniapedia ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P6200'), ('prop', 'BBC News topic ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1617'), ('prop', 'BBC Things ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('prop', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2390'), ('prop', 'Ballotpedia ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P268'), ('prop', 'Bibliothèque nationale de France ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P373'), ('prop', 'Commons category')]
[('pr', 'http://www.wikidata.org/prop/direct/P935'), ('prop', 'Commons gallery')]
[('pr', 'http://www.wikidata.org/prop/direct/P3509'), ('prop', 'Dagens Nyheter topic ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('prop', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P646'), ('prop', 'Freebase ID')]
[('pr', 'http://www.wikidata.org/prop/direct/

72

In [18]:
#filter by italian to focus on few result
queryString = """
SELECT DISTINCT ?o

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?y ?p.
?z wdt:P1545 ?o.
?z sc:name?obj .

filter(regex(?obj,".*talian.*"))
}
ORDER BY ?prop
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [19]:
#filter by italian to focus on few result
queryString = """
SELECT DISTINCT ?obj ?o

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?y ?p.
?z wdt:P580 ?o.
?z sc:name?obj .

filter(regex(?obj,".*talian.*"))
}
ORDER BY ?prop
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('obj', '2013 Italian presidential election'), ('o', '2013-04-18T00:00:00Z')]
[('obj', '2006 Italian presidential election'), ('o', '2006-05-08T00:00:00Z')]
[('obj', '1992 Italian presidential election'), ('o', '1992-05-13T00:00:00Z')]
[('obj', '1948 Italian presidential election'), ('o', '1948-05-10T00:00:00Z')]
[('obj', '1955 Italian presidential election'), ('o', '1955-04-28T00:00:00Z')]
[('obj', '1962 Italian presidential election'), ('o', '1962-05-02T00:00:00Z')]
[('obj', '2015 Italian presidential election'), ('o', '2015-01-29T00:00:00Z')]
[('obj', '1971 Italian presidential election'), ('o', '1971-12-09T00:00:00Z')]
[('obj', '1978 Italian presidential election'), ('o', '1978-06-29T00:00:00Z')]
[('obj', '1964 Italian presidential election'), ('o', '1964-12-16T00:00:00Z')]


10

In [20]:
#limit to 1 to take the first
queryString = """
SELECT ?country ?obj

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?pr ?p.
?z wdt:P17 ?y.
?z sc:name?obj .
?y sc:name?country .

filter(regex(?obj,".*talian.*"))
}
ORDER BY ?obj
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('country', 'Italy'), ('obj', '1948 Italian presidential election')]


1

In [22]:
#print other classes of elections
queryString = """

SELECT DISTINCT ?y ?prop
WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z wdt:P31 ?y.

?z sc:name?obj .
?y sc:name?prop .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q106691645'), ('prop', 'Argentine general election')]
[('y', 'http://www.wikidata.org/entity/Q15146717'), ('prop', 'Italian presidential election')]
[('y', 'http://www.wikidata.org/entity/Q106690375'), ('prop', 'Zambian general election')]
[('y', 'http://www.wikidata.org/entity/Q47516525'), ('prop', 'Cuban parliamentary election')]
[('y', 'http://www.wikidata.org/entity/Q16361747'), ('prop', 'Latvian presidential election')]
[('y', 'http://www.wikidata.org/entity/Q22696400'), ('prop', 'Pakistani general election')]
[('y', 'http://www.wikidata.org/entity/Q22276119'), ('prop', 'Serbian general election')]
[('y', 'http://www.wikidata.org/entity/Q24397514'), ('prop', 'United States House of Representatives election')]
[('y', 'http://www.wikidata.org/entity/Q19801853'), ('prop', 'Republic of China Presidential and Vice Presidential election')]
[('y', 'http://www.wikidata.org/entity/Q25343628'), ('prop', 'Costa Rican parliamentary election')]
[(

90

Presidential election : `wd:Q858439`

In [42]:
queryString = """
SELECT ?country ?name ?birth ?obj

WHERE{
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?a wdt:P569 ?birth.
?z ?pr ?p.
?z wdt:P31/wdt:P279 wd:Q858439.
?z wdt:P580 ?dateExt.
?z wdt:P17 ?y.

filter(?dateExt = ?min)
        
    {
        SELECT ?y min(?date) AS ?min

        WHERE { 
        ?p wdt:P279* wd:Q248577.
        ?a wdt:P39 ?p.
        ?z wdt:P991 ?a.
        ?z ?pr ?p.
        ?z wdt:P580 ?date.
        ?z wdt:P17 ?y.
        ?z wdt:P31/wdt:P279 wd:Q858439.
        
        
        }
        GROUP BY ?y
    }
    
?z sc:name?obj .
?a sc:name?name .
?y sc:name?country .

}LIMIT 10



"""

print("Results")
run_query(queryString)

Results
[('country', 'United States of America'), ('name', 'George Washington'), ('birth', '1732-02-22T00:00:00Z'), ('obj', '1788–89 United States presidential election')]
[('country', 'Italy'), ('name', 'Luigi Einaudi'), ('birth', '1874-03-24T00:00:00Z'), ('obj', '1948 Italian presidential election')]


2

In [43]:
queryString = """
SELECT ?country ?obj ?date

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?pr ?p.
?z wdt:P17 ?y.
?z sc:name?obj .
?y sc:name?country .
BIND (SUBSTR(str(?obj), 0,4) AS ?date)
filter(regex(?obj,".*talian.*"))
}
ORDER BY ?obj
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'Italy'), ('obj', '1948 Italian presidential election'), ('date', '1948')]
[('country', 'Italy'), ('obj', '1955 Italian presidential election'), ('date', '1955')]
[('country', 'Italy'), ('obj', '1962 Italian presidential election'), ('date', '1962')]
[('country', 'Italy'), ('obj', '1964 Italian presidential election'), ('date', '1964')]
[('country', 'Italy'), ('obj', '1971 Italian presidential election'), ('date', '1971')]
[('country', 'Italy'), ('obj', '1978 Italian presidential election'), ('date', '1978')]
[('country', 'Italy'), ('obj', '1985 Italian presidential election'), ('date', '1985')]
[('country', 'Italy'), ('obj', '1992 Italian presidential election'), ('date', '1992')]
[('country', 'Italy'), ('obj', '1999 Italian presidential election'), ('date', '1999')]
[('country', 'Italy'), ('obj', '2006 Italian presidential election'), ('date', '2006')]


10

In [44]:
queryString = """
SELECT ?country ?name ?birth ?obj

WHERE{
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?a wdt:P569 ?birth.
?z ?pr ?p.
?z wdt:P31/wdt:P279 wd:Q858439.
?z wdt:P580 ?dateExt.
?z wdt:P17 ?x.
filter(?dateExt = ?min)
        
    {
        SELECT ?y min(?date) AS ?min

        WHERE { 
        ?p wdt:P279* wd:Q248577.
        ?a wdt:P39 ?p.
        ?z wdt:P991 ?a.
        ?z ?pr ?p.
        ?z wdt:P17 ?y.
        ?z wdt:P580 ?date.
        ?z wdt:P31/wdt:P279 wd:Q858439.
        ?z sc:name?obj .
        
        
        }
        GROUP BY ?y
    }
    
?z sc:name?obj .
?a sc:name?name .
?x sc:name?country .

}LIMIT 10



"""

print("Results")
run_query(queryString)

Results
[('country', 'United States of America'), ('name', 'George Washington'), ('birth', '1732-02-22T00:00:00Z'), ('obj', '1788–89 United States presidential election')]
[('country', 'Italy'), ('name', 'Luigi Einaudi'), ('birth', '1874-03-24T00:00:00Z'), ('obj', '1948 Italian presidential election')]


2

Only USA and Italy have the start time.. The others have the year in the name.

In [23]:
#use the substring to extract the year from the name
queryString = """
SELECT ?y ?country min(?date) AS ?min

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?pr ?p.
?z wdt:P17 ?y.
?z wdt:P31/wdt:P279 wd:Q858439.
?z sc:name?obj .
BIND (SUBSTR(str(?obj), 0,4) AS ?date)

?y sc:name?country .

}
GROUP BY ?y ?country 
        
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q155'), ('country', 'Brazil'), ('min', 'Braz')]
[('y', 'http://www.wikidata.org/entity/Q419'), ('country', 'Peru'), ('min', '1894')]
[('y', 'http://www.wikidata.org/entity/Q334'), ('country', 'Singapore'), ('min', '1993')]
[('y', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy'), ('min', '1948')]
[('y', 'http://www.wikidata.org/entity/Q1036'), ('country', 'Uganda'), ('min', '2016')]
[('y', 'http://www.wikidata.org/entity/Q211'), ('country', 'Latvia'), ('min', '2011')]
[('y', 'http://www.wikidata.org/entity/Q403'), ('country', 'Serbia'), ('min', '1997')]
[('y', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria'), ('min', '1963')]
[('y', 'http://www.wikidata.org/entity/Q45'), ('country', 'Portugal'), ('min', '1976')]
[('y', 'http://www.wikidata.org/entity/Q928'), ('country', 'Philippines'), ('min', '1935')]
[('y', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('min', '1949')]
[('y', 'http://www.wikidata.

19

In [24]:
#Brazil and Zimbawe not start with the year
queryString = """
SELECT ?y ?country ?obj

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?pr ?p.
?z wdt:P17 ?y.
?z wdt:P31/wdt:P279 wd:Q858439.
?z sc:name?obj .

filter(regex(?country,".*imbab.*") || regex(?country,".*razil.*"))
?y sc:name?country .

}

        
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q155'), ('country', 'Brazil'), ('obj', 'Brazilian presidential election, 2014')]
[('y', 'http://www.wikidata.org/entity/Q954'), ('country', 'Zimbabwe'), ('obj', 'Zimbabwean presidential election, 2013')]


2

In [25]:
#remove brazil and zimbawe from the result set
queryString = """
SELECT ?country min(?date) AS ?min

WHERE { 
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?z ?pr ?p.
?z wdt:P17 ?y.
?z wdt:P31/wdt:P279 wd:Q858439.

?z sc:name?obj .
?y sc:name?country .

BIND (SUBSTR(str(?obj), 0,4) AS ?date)

FILTER(!(regex(?country,".*imbab.*") || regex(?country,".*razil.*")))

}
GROUP BY ?y ?country
        
"""

print("Results")
run_query(queryString)

Results
[('country', 'Portugal'), ('min', '1976')]
[('country', 'Zambia'), ('min', '2008')]
[('country', 'Colombia'), ('min', '1914')]
[('country', 'Germany'), ('min', '1949')]
[('country', 'Peru'), ('min', '1894')]
[('country', 'Italy'), ('min', '1948')]
[('country', 'Poland'), ('min', '2010')]
[('country', 'Latvia'), ('min', '2011')]
[('country', 'United States of America'), ('min', '1788')]
[('country', 'Taiwan'), ('min', '1948')]
[('country', 'Philippines'), ('min', '1935')]
[('country', 'Uganda'), ('min', '2016')]
[('country', 'Serbia'), ('min', '1997')]
[('country', 'Singapore'), ('min', '1993')]
[('country', 'Austria'), ('min', '1963')]
[('country', 'Argentina'), ('min', '1826')]
[('country', 'South Korea'), ('min', '1948')]


17

In [48]:
queryString = """
SELECT ?country ?name ?birth

WHERE{
?p wdt:P279* wd:Q248577.
?a wdt:P39 ?p.
?z wdt:P991 ?a.
?a wdt:P569 ?birth.
?z ?pr ?p.
?z wdt:P31/wdt:P279 wd:Q858439.
?z wdt:P17 ?x.

?z sc:name?obj.
BIND (SUBSTR(str(?obj), 0,4) AS ?dateX)

filter(?x = ?y and ?dateX = ?min)
        
    {
        SELECT ?y min(?date) AS ?min

        WHERE { 
        ?p wdt:P279* wd:Q248577.
        ?a wdt:P39 ?p.
        ?z wdt:P991 ?a.
        ?z ?pr ?p.
        ?z wdt:P17 ?y.
        ?z wdt:P31/wdt:P279 wd:Q858439.
        
        ?z sc:name?obj .
        ?y sc:name?country .
        
        BIND (SUBSTR(str(?obj), 0,4) AS ?date)
        
        FILTER(!(regex(?country,".*imbab.*") || regex(?country,".*razil.*")))
        
        }
        GROUP BY ?y
    }
     .
?a sc:name?name .
?x sc:name?country .

}LIMIT 40



"""

print("Results")
run_query(queryString)

Results
[('country', 'Singapore'), ('name', 'Ong Teng Cheong'), ('birth', '1936-01-22T00:00:00Z')]
[('country', 'Uganda'), ('name', 'Yoweri Museveni'), ('birth', '1944-09-15T00:00:00Z')]
[('country', 'Poland'), ('name', 'Bronisław Komorowski'), ('birth', '1952-06-04T00:00:00Z')]
[('country', 'Taiwan'), ('name', 'Chiang Kai-shek'), ('birth', '1887-10-31T00:00:00Z')]
[('country', 'United States of America'), ('name', 'George Washington'), ('birth', '1732-02-22T00:00:00Z')]
[('country', 'Peru'), ('name', 'Andrés Avelino Cáceres'), ('birth', '1836-11-10T00:00:00Z')]
[('country', 'Philippines'), ('name', 'Manuel L. Quezon'), ('birth', '1878-08-19T00:00:00Z')]
[('country', 'Zambia'), ('name', 'Rupiah Banda'), ('birth', '1937-01-01T00:00:00Z')]
[('country', 'Zambia'), ('name', 'Rupiah Banda'), ('birth', '1937-02-13T00:00:00Z')]
[('country', 'Germany'), ('name', 'Theodor Heuss'), ('birth', '1884-01-31T00:00:00Z')]
[('country', 'Latvia'), ('name', 'Andris Bērziņš'), ('birth', '1944-12-10T00:00:

19

Not a good result. But I tried everything.

### Task 3.2

Group by country to find the number of president of each country

In [49]:
queryString = """
SELECT ?country count(DISTINCT ?a) AS ?count
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?p wdt:P17 ?c.
?c sc:name?country .

} 
GROUP BY ?c ?country
ORDER BY DESC (?count)
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('country', 'Honduras'), ('count', '84')]
[('country', 'United States of America'), ('count', '64')]
[('country', 'Colombia'), ('count', '60')]
[('country', 'Argentina'), ('count', '59')]
[('country', 'Mexico'), ('count', '59')]
[('country', 'Venezuela'), ('count', '58')]
[('country', 'Paraguay'), ('count', '57')]
[('country', 'Haiti'), ('count', '55')]
[('country', 'Peru'), ('count', '53')]
[('country', 'Guatemala'), ('count', '53')]
[('country', 'Brazil'), ('count', '42')]
[('country', 'Costa Rica'), ('count', '40')]
[('country', 'Chile'), ('count', '37')]
[('country', 'Cuba'), ('count', '30')]
[('country', 'El Salvador'), ('count', '28')]
[('country', 'Syria'), ('count', '25')]
[('country', 'Federal Republic of Central America'), ('count', '25')]
[('country', 'Lebanon'), ('count', '22')]
[('country', 'Taiwan'), ('count', '21')]
[('country', 'Portugal'), ('count', '19')]
[('country', 'Poland'), ('count', '19')]
[('country', 'Serbia'), ('count', '18')]
[('country', 'Comoros')

99

## Task 4 
Is there a country that had at some point a woman as a president?

In [26]:
#print the genders
queryString = """
SELECT DISTINCT ?o ?gender
WHERE { 
?a wdt:P21 ?o.
?o sc:name?gender .

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q301702'), ('gender', 'two-spirit')]
[('o', 'http://www.wikidata.org/entity/Q1289754'), ('gender', 'neutrois')]
[('o', 'http://www.wikidata.org/entity/Q505371'), ('gender', 'agender')]
[('o', 'http://www.wikidata.org/entity/Q52261234'), ('gender', 'neutral sex')]
[('o', 'http://www.wikidata.org/entity/Q24238356'), ('gender', 'unknown')]
[('o', 'http://www.wikidata.org/entity/Q2449503'), ('gender', 'transgender male')]
[('o', 'http://www.wikidata.org/entity/Q3177577'), ('gender', 'muxe')]
[('o', 'http://www.wikidata.org/entity/Q48279'), ('gender', 'third gender')]
[('o', 'http://www.wikidata.org/entity/Q18116794'), ('gender', 'genderfluid')]
[('o', 'http://www.wikidata.org/entity/Q6581072'), ('gender', 'female')]
[('o', 'http://www.wikidata.org/entity/Q6581097'), ('gender', 'male')]
[('o', 'http://www.wikidata.org/entity/Q1052281'), ('gender', 'transgender female')]
[('o', 'http://www.wikidata.org/entity/Q15145778'), ('gender', 'cisgender m

44

ASK query to answer at the question.

In [51]:
queryString = """
ASK
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?p wdt:P17 ?c.
?a wdt:P21 ?g.
?c sc:name?country .
?g sc:name?gender .
FILTER (REGEX(?gender,\".*female.*\"))

}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

Since ASK reply TRUE, I print some example just for curiosity


In [52]:
queryString = """
SELECT DISTINCT ?name ?country ?gender
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?p wdt:P17 ?c.
?a wdt:P21 ?g.
?c sc:name?country .
?a sc:name?name .
?g sc:name?gender .
FILTER (REGEX(?gender,\".*female.*\"))

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('name', 'Ameenah Gurib-Fakim'), ('country', 'Mauritius'), ('gender', 'female')]
[('name', 'Monique Ohsan Bellepeau'), ('country', 'Mauritius'), ('gender', 'female')]
[('name', 'Sahle-Work Zewde'), ('country', 'Ethiopia'), ('gender', 'female')]
[('name', 'Vjosa Osmani'), ('country', 'Kosovo'), ('gender', 'female')]
[('name', 'Atifete Jahjaga'), ('country', 'Kosovo'), ('gender', 'female')]
[('name', 'Dilma Rousseff'), ('country', 'Brazil'), ('gender', 'female')]
[('name', 'Vigdís Finnbogadóttir'), ('country', 'Iceland'), ('gender', 'female')]
[('name', 'Kersti Kaljulaid'), ('country', 'Estonia'), ('gender', 'female')]
[('name', 'Vaira Vīķe-Freiberga'), ('country', 'Latvia'), ('gender', 'female')]
[('name', 'Maia Sandu'), ('country', 'Moldova'), ('gender', 'female')]
[('name', 'Ema Derossi-Bjelajac'), ('country', 'Croatia'), ('gender', 'female')]
[('name', 'Kolinda Grabar-Kitarović'), ('country', 'Croatia'), ('gender', 'female')]
[('name', 'Biljana Plavšić'), ('country', 'Bosnia

48

Do again the query with the constraint that the holder of the title of President Of Republic must be a Human.

In [53]:
queryString = """
SELECT DISTINCT ?name ?country ?gender
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?p wdt:P17 ?c.
?a wdt:P21 ?g.
?c sc:name?country .
?a sc:name?name .
?g sc:name?gender .
FILTER (REGEX(?gender,\".*female.*\"))

} 
LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('name', 'Ameenah Gurib-Fakim'), ('country', 'Mauritius'), ('gender', 'female')]
[('name', 'Monique Ohsan Bellepeau'), ('country', 'Mauritius'), ('gender', 'female')]
[('name', 'Sahle-Work Zewde'), ('country', 'Ethiopia'), ('gender', 'female')]
[('name', 'Vjosa Osmani'), ('country', 'Kosovo'), ('gender', 'female')]
[('name', 'Atifete Jahjaga'), ('country', 'Kosovo'), ('gender', 'female')]
[('name', 'Dilma Rousseff'), ('country', 'Brazil'), ('gender', 'female')]
[('name', 'Vigdís Finnbogadóttir'), ('country', 'Iceland'), ('gender', 'female')]
[('name', 'Kersti Kaljulaid'), ('country', 'Estonia'), ('gender', 'female')]
[('name', 'Vaira Vīķe-Freiberga'), ('country', 'Latvia'), ('gender', 'female')]
[('name', 'Maia Sandu'), ('country', 'Moldova'), ('gender', 'female')]
[('name', 'Ema Derossi-Bjelajac'), ('country', 'Croatia'), ('gender', 'female')]
[('name', 'Kolinda Grabar-Kitarović'), ('country', 'Croatia'), ('gender', 'female')]
[('name', 'Biljana Plavšić'), ('country', 'Bosnia

45

Found something strange!!  Maybe also people from tv series or films is considered as president! (Mr. Garrison)

## Task 5
Analyze the number of presidents per country through time

5.1 What are the top-5 countries for number of presidents? Which countries had the least?

5.2 For how many presidents we know a date of death in each country?

5.3 Which are the professions of different presidents? How many presidents had a specific profession?

In [27]:
#Find the top-5 counties
queryString = """
SELECT ?country count(DISTINCT ?a) AS ?count
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p.
?p wdt:P17 ?c.
?c sc:name?country .

} 
GROUP BY ?c ?country
ORDER BY DESC (?count)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('country', 'Honduras'), ('count', '84')]
[('country', 'United States of America'), ('count', '64')]
[('country', 'Colombia'), ('count', '60')]
[('country', 'Argentina'), ('count', '59')]
[('country', 'Mexico'), ('count', '59')]


5

In [28]:
#Find the countries with the least number of president
queryString = """
SELECT ?country ?count
WHERE
{
    {
        SELECT min(?count) AS ?min_c
        WHERE
        {
            SELECT ?country count(DISTINCT ?a) AS ?count
            WHERE { 

            ?p wdt:P279+ wd:Q248577 . 
            ?a wdt:P39 ?p.
            ?p wdt:P17 ?c.
            ?c sc:name?country .

            } 
            GROUP BY ?c ?country
        }
    }
    {
        SELECT ?country count(DISTINCT ?a) AS ?count
        WHERE { 

        ?p wdt:P279+ wd:Q248577 . 
        ?a wdt:P39 ?p.
        ?p wdt:P17 ?c.
        ?c sc:name?country .

        } 
        GROUP BY ?c ?country
    }
    FILTER(?count=?min_c)
}
"""

print("Results")
run_query(queryString)

Results
[('country', 'Confederate States of America'), ('count', '1')]
[('country', 'Republic of Kosova'), ('count', '1')]
[('country', 'Spanish Empire'), ('count', '1')]


3

In [29]:
#Do the query again with the constraint that I looking just for a human.
queryString = """
SELECT ?country count(DISTINCT ?a) AS ?count
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p;
    wdt:P31 wd:Q5.
?p wdt:P17 ?c.
?c sc:name?country .

} 
GROUP BY ?c ?country
ORDER BY DESC (?count)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('country', 'Honduras'), ('count', '83')]
[('country', 'Colombia'), ('count', '60')]
[('country', 'Argentina'), ('count', '59')]
[('country', 'Mexico'), ('count', '59')]
[('country', 'Venezuela'), ('count', '58')]


5

Found that United States of America as a lot of "imaginary" president. In fact my intuition was born since I noticed this strange number of president of United States, 64. And running again the query I found 45 that is the right number of president of United States in the history.

Anyway I will not keep this constraint for the other query.

Find how many president we know the date of death for each country

In [57]:
queryString = """
SELECT ?country count(DISTINCT ?a) AS ?num_of_death
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p;
    wdt:P570 ?o.
?p wdt:P17 ?c.

?c sc:name?country .
}
GROUP BY ?country
ORDER BY DESC (?num_of_death)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('country', 'Honduras'), ('num_of_death', '72')]
[('country', 'Venezuela'), ('num_of_death', '55')]
[('country', 'Colombia'), ('num_of_death', '54')]
[('country', 'Mexico'), ('num_of_death', '52')]
[('country', 'Argentina'), ('num_of_death', '51')]
[('country', 'Paraguay'), ('num_of_death', '49')]
[('country', 'Haiti'), ('num_of_death', '48')]
[('country', 'Peru'), ('num_of_death', '44')]
[('country', 'Guatemala'), ('num_of_death', '43')]
[('country', 'United States of America'), ('num_of_death', '39')]
[('country', 'Brazil'), ('num_of_death', '35')]
[('country', 'Chile'), ('num_of_death', '34')]
[('country', 'Costa Rica'), ('num_of_death', '33')]
[('country', 'Cuba'), ('num_of_death', '28')]
[('country', 'Syria'), ('num_of_death', '24')]
[('country', 'Federal Republic of Central America'), ('num_of_death', '22')]
[('country', 'El Salvador'), ('num_of_death', '21')]
[('country', 'Taiwan'), ('num_of_death', '17')]
[('country', 'Portugal'), ('num_of_death', '16')]
[('country', '

20

Find the professions of different presidents.

In [58]:
queryString = """
SELECT DISTINCT ?prof ?profession
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p;
    wdt:P106 ?prof.

?prof sc:name?profession .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('prof', 'http://www.wikidata.org/entity/Q2267418'), ('profession', 'industrial engineer')]
[('prof', 'http://www.wikidata.org/entity/Q26267537'), ('profession', 'serial rapist')]
[('prof', 'http://www.wikidata.org/entity/Q12013238'), ('profession', 'ophthalmologist')]
[('prof', 'http://www.wikidata.org/entity/Q1569495'), ('profession', 'lecturer')]
[('prof', 'http://www.wikidata.org/entity/Q183318'), ('profession', 'dictator')]
[('prof', 'http://www.wikidata.org/entity/Q202883'), ('profession', 'veterinarian')]
[('prof', 'http://www.wikidata.org/entity/Q212238'), ('profession', 'civil servant')]
[('prof', 'http://www.wikidata.org/entity/Q2159907'), ('profession', 'criminal')]
[('prof', 'http://www.wikidata.org/entity/Q3264451'), ('profession', 'cardiologist')]
[('prof', 'http://www.wikidata.org/entity/Q47227062'), ('profession', 'agitator')]
[('prof', 'http://www.wikidata.org/entity/Q83307'), ('profession', 'minister')]
[('prof', 'http://www.wikidata.org/entity/Q849424'), ('p

20

How many presidents had a specific profession?

In [59]:
queryString = """
SELECT ?a ?name (count(DISTINCT ?prof) AS ?num_of_prof)
WHERE { 

?p wdt:P279+ wd:Q248577 . 
?a wdt:P39 ?p;
    wdt:P106 ?prof.

?prof sc:name?profession .
?a sc:name?name .
}
GROUP BY ?a ?name
HAVING (count(DISTINCT ?prof) =1)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q436743'), ('name', 'José María Peralta'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q57452'), ('name', 'François Bozizé'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q290061'), ('name', 'Veerasamy Ringadoo'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q755794'), ('name', 'Ati George Sokomanu'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q5939792'), ('name', 'Francisco Gómez'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q320103'), ('name', 'Souphanouvong'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q537067'), ('name', 'Jamtsangiin Damdinsüren'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q5220343'), ('name', 'Danny Faure'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q11613'), ('name', 'Harry S Truman'), ('num_of_prof', '1')]
[('a', 'http://www.wikidata.org/entity/Q449421'), ('name', 'Jean-Pierre 

100